# How to deploy ADK Apps to Agent Engine

[Documentation link 🔗](https://google.github.io/adk-docs/deploy/agent-engine/)

## 1. Set the local environment variables
These will be used in the Agent Engine deployment

In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../trends_and_insights_agent/.env")  # take environment variables

True

## 2. Set up the `AdkApp` Convenience wrapper for ADK Agents -> Agent Engine

Note the injection of the environment variables.

In [2]:
from vertexai.preview.reasoning_engines import AdkApp
from google.adk.artifacts import GcsArtifactService
from google.adk.sessions import VertexAiSessionService

from trends_and_insights_agent import agent

env_vars = {}

env_vars["GOOGLE_GENAI_USE_VERTEXAI"] = os.getenv("GOOGLE_GENAI_USE_VERTEXAI")
env_vars["BUCKET"] = os.getenv("BUCKET", "gs://default-bucket")
env_vars["GOOGLE_CLOUD_PROJECT_NUMBER"] = os.getenv("GOOGLE_CLOUD_PROJECT_NUMBER")
env_vars["YT_SECRET_MNGR_NAME"] = os.getenv("YT_SECRET_MNGR_NAME")
# env_vars["SESSION_STATE_JSON_PATH"] = os.getenv("SESSION_STATE_JSON_PATH")


my_agent = AdkApp(
    agent=agent.root_agent,
    enable_tracing=True,
    env_vars=env_vars,
)

INFO:root:

`SESSION_STATE_JSON_PATH`: None


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
/Users/jwortz/repos/zghost/zghost/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/jwortz/repos/zghost/zghost/trends_and_insights_agent/common_agents/trend_assistant/tools.py:153: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return max_date.iloc[0][0].strftime("%m/%d/%Y")
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


# 3. Test the Agent Engine locally

In [3]:
session = my_agent.create_session(user_id="jwortz", session_id="12345")
session

Session(id='12345', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1753822682.469465)

In [4]:
my_agent.list_sessions(user_id="jwortz")

ListSessionsResponse(sessions=[Session(id='12345', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1753822682.469465)])

In [9]:
from pprint import pprint


def stream_agent(agent, prompt: str, session) -> None | Exception:
    try:
        session_id = session["id"]
    # local prefers attributes vs. remote prefers dict keys
    except TypeError:
        session_id = session.id
    except Exception as e:
        return f"Session Object not valid: {e}"

    for event in agent.stream_query(
        user_id="jwortz",
        session_id=session_id,
        message=prompt,
    ):
        pprint(event)

In [6]:
first_prompt = f"Hello"
stream_agent(my_agent, first_prompt, session)

INFO:root:

Loading Initial State (empty): {'state': {'final_select_ad_copies': {'final_select_ad_copies': []}, 'final_select_vis_concepts': {'final_select_vis_concepts': []}, 'img_artifact_keys': {'img_artifact_keys': []}, 'vid_artifact_keys': {'vid_artifact_keys': []}, 'brand': '', 'target_product': '', 'target_audience': '', 'key_selling_points': '', 'target_search_trends': {'target_search_trends': []}, 'target_yt_trends': {'target_yt_trends': []}}}


INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
INFO:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:

You are a helpful AI assistant, part of a multi-agent system designed for advanced web research and ad creative generation.
Do not perform any research yourself. Your job is to **delegate**.


You are an Expert AI Marketing Research & Strategy Assistant. 

Yo

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'_state_init': True,
                             'brand': '',
                             'final_select_ad_copies': {'final_select_ad_copies': []},
                             'final_select_vis_concepts': {'final_select_vis_concepts': []},
                             'gcs_folder': '2025_07_29_20_58',
                             'img_artifact_keys': {'img_artifact_keys': []},
                             'key_selling_points': '',
                             'target_audience': '',
                             'target_product': '',
                             'target_search_trends': {'target_search_trends': []},
                             'target_yt_trends': {'target_yt_trends': []},
                             'vid_artifact_keys': {'vid_artifact_keys': []}}},
 'author': 'root_agent',
 'id': '3cc14e05-2e5b-44ba-b813-46731215579a',
 'invocation_id': 'e-93e1dd23-03ec-4958-ac4

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Hello! I'm your AI Marketing Research & Strategy Assistant. I can help you with comprehensive insights, trend analysis, and creative ideas for your marketing campaigns.

To get started, I'll transfer you to my `trends_and_insights_agent` sub-agent, who will help us capture your campaign metadata and identify interesting trends.

-----------------------------------------------------------
Function calls:
name: transfer_to_agent, args: {'agent_name': 'trends_and_insights_agent'}
-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Referer","Content-Encoding":"gzip","Date":"Tue, 29 Jul 2025 20:58:11 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Options":"SAMEORIGIN","X-Content-Type-Options":"nosniff","Alt-Svc":"h3=\":443\"; ma=

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'request_count': 1,
                             'timer_start': 1753822688.412184}},
 'author': 'root_agent',
 'content': {'parts': [{'text': "Hello! I'm your AI Marketing Research & "
                                'Strategy Assistant. I can help you with '
                                'comprehensive insights, trend analysis, and '
                                'creative ideas for your marketing campaigns.\n'
                                '\n'
                                "To get started, I'll transfer you to my "
                                '`trends_and_insights_agent` sub-agent, who '
                                'will help us capture your campaign metadata '
                                'and identify interesting trends.\n'},
                       {'function_call': {'args': {'agent_name': 'trends_and_insights_agent'},
                                      

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Hello! I'm your Trends and Insights Assistant. I can help you by gathering information for your marketing campaign.

To start, please tell me the following:
*   **Brand Name:**
*   **Target Audience:**
*   **Target Product:**
*   **Key Selling Points of the product:**
-----------------------------------------------------------
Function calls:

-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Referer","Content-Encoding":"gzip","Date":"Tue, 29 Jul 2025 20:58:13 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Options":"SAMEORIGIN","X-Content-Type-Options":"nosniff","Alt-Svc":"h3=\":443\"; ma=2592000,h3-29=\":443\"; ma=2592000","Transfer-Encoding":"chunked"}},"candidates":[{"content":{"parts":[{"text":"Hello! I'm your Trends a

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'text': "Hello! I'm your Trends and Insights "
                                'Assistant. I can help you by gathering '
                                'information for your marketing campaign.\n'
                                '\n'
                                'To start, please tell me the following:\n'
                                '*   **Brand Name:**\n'
                                '*   **Target Audience:**\n'
                                '*   **Target Product:**\n'
                                '*   **Key Selling Points of the product:**'}],
             'role': 'model'},
 'id': 'f7798729-2f6a-4c9d-9cd5-006f955e8c7e',
 'invocation_id': 'e-93e1dd23-03ec-4958-ac4a-9c043c66ab75',
 'timestamp': 1753822691.101807,
 'usage_metadata': {'candidates_token_count': 65,
                    'candidates_toke

In [ ]:
followup_prompt = "Create images from these great ideas!"
stream_agent(my_agent, followup_prompt, session)

## 4. Deploy to Vertex AI Agent Engine
To call and manage agents in production, deploy the agent to Vertex AI Agent Engine.

**Important - run `poetry build` to package the agent**

In [5]:
! poetry build --format=wheel --output=notebooks/deployment

Building trends_and_insights_agent (0.1.0)
  - Building wheel
  - Built trends_and_insights_agent-0.1.0-py3-none-any.whl


#### Initialize the Vertex client, then create a `remote_agent` that is deployed to Vertex

This also takes the packaged agent code. This is required for more complex agents that have nested dependencies and require packaging

Also, before running - be sure to give Secret Manager access to the Agent Engine service account. This can simply be done by running `. setup_ae_sm_access.sh`.

The code can also be ran as follows (relative to repo root):

```bash
source trends_and_insights_agent/.env

export RE_SA="service-${GOOGLE_CLOUD_PROJECT_NUMBER}@gcp-sa-aiplatform-re.iam.gserviceaccount.com"
gcloud secrets add-iam-policy-binding "projects/$GOOGLE_CLOUD_PROJECT/secrets/$YT_SECRET_MNGR_NAME" \
  --member="serviceAccount:$RE_SA" \
  --role="roles/secretmanager.secretAccessor"
```

In [4]:
from vertexai import agent_engines  # optional cleanup

for agent in agent_engines.list():
    agent.delete(force=True)

In [6]:
import vertexai
from vertexai import agent_engines

GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")
BUCKET = os.getenv("BUCKET")

vertexai.init(
    project=GOOGLE_CLOUD_PROJECT,
    location="us-central1",
    staging_bucket=BUCKET,
)

remote_agent = agent_engines.create(
    agent_engine=my_agent,
    display_name="trends-and-insights",
    description="You are a helpful AI assistant, part of a multi-agent system designed for advanced web research and ad creative generation.",
    requirements=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
        "db-dtypes",
        "tabulate",
    ],
    extra_packages=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
        "installation_scripts/install_opencv.sh",
        "installation_scripts/install_ffmpeg.sh",
    ],
    env_vars=env_vars,
    build_options={
        "installation": [
            "installation_scripts/install_opencv.sh",
            "installation_scripts/install_ffmpeg.sh",
        ]
    },
)

Identified the following requirements: {'pydantic': '2.11.7', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.105.0'}


INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.105.0'}


Failed to parse constraint: deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl. Exception: Expected end or semicolon (after name and no valid version specifier)
    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


The following requirements are missing: {'pydantic', 'cloudpickle', 'google-cloud-aiplatform'}


...

INFO:vertexai.agent_engines:AgentEngine created. Resource name: projects/679926387543/locations/us-central1/reasoningEngines/3252927141007851520


To use this AgentEngine in another session:


INFO:vertexai.agent_engines:To use this AgentEngine in another session:


agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/3252927141007851520')


INFO:vertexai.agent_engines:agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/3252927141007851520')


### Save the agent engine resource name

This can be used by the command to deploy it to Agentspace

```bash
agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/1093257605637210112')
```

## 5. Try it remotely

In [7]:
online_session = remote_agent.create_session(user_id="jwortz")
online_session

{'lastUpdateTime': 1754493816.518254,
 'events': [],
 'userId': 'jwortz',
 'appName': '3252927141007851520',
 'id': '2675871003157987328',
 'state': {}}

#### Same idea above applies to remote agents

In [10]:
first_prompt = f"Hello"

stream_agent(remote_agent, first_prompt, online_session)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'_state_init': True,
                             'brand': '',
                             'final_select_ad_copies': {'final_select_ad_copies': []},
                             'final_select_vis_concepts': {'final_select_vis_concepts': []},
                             'gcs_folder': '2025_08_06_15_23',
                             'img_artifact_keys': {'img_artifact_keys': []},
                             'key_selling_points': '',
                             'target_audience': '',
                             'target_product': '',
                             'target_search_trends': {'target_search_trends': []},
                             'target_yt_trends': {'target_yt_trends': []},
                             'vid_artifact_keys': {'vid_artifact_keys': []}}},
 'author': 'root_agent',
 'id': 'd25979c5-5eb8-477a-ad87-14f5e0150bee',
 'invocation_id': 'e-d6b5a3bb-803a-4bf3-99d

# 6. Optional Cleanup

In [ ]:
remote_agent.delete(force=True)